# Lecture 4 - The workflow of deep learning

In the previous lectures we have imported the correct dataset from keras.datasets and started fitting some deep learning models. In the real world, this is often not the case and we’ll have to start from scratch. Therefore we need a universal step-by-step blueprint that we can use to approach and solve any machine (deep) learning problem. This workflow can be structured in three parts:
1. **Define the task** — Understand the problem domain and the business logic underlying what the customer asked for. Collect a dataset, understand what the data represents, and choose how we will measure success on the task.
2. **Develop a model** — Prepare data so that it can be processed by a machine learning model, select a model evaluation protocol and a simple baseline to beat, train a first model that has generalization power and that can overfit, and then regularize and tune the model until we achieve the best possible generalization performance.
3. **Deploy the model** — Present your work to stakeholders, ship the model to a web server, a mobile app, a web page, or an embedded device, monitor the model’s performance in the wild, and start collecting the data you’ll need to build the next-generation model.

## Defining the task
This stage consists in a deep understanding of the context of a task and includes the following steps:
1. Framing the problem
2. Collecting a dataset
3. Understanding data
4. Choosing a measure of success

### Framing the problem
usually involves many detailed discussions with stakeholders. We need to consider the following questions:
- What will our input data be? 
- What are we trying to predict?
- What type of machine learning task are you facing?
- What do existing solutions look like?
- Are there particular constraints we will need to deal with?
You have to be aware of the fact that not all problems can be solved with machine learning and sometimes machine learning isn’t  the best way to make sense of the data, and you should use something else, such as plain old-school statistical analysis.

### Collecting a dataset
Data collection is the most arduous, time-consuming, and costly part of most machine learning projects. A model’s ability to generalize comes almost entirely from the properties of the data it is trained on: the number of data points, the reliability of labels, and the quality of features. Therefore a good dataset is an asset worthy of care and investment. We need to keep in mind the following issues:
1. **Annotations** - If we’re doing supervised learning, then once we’ve collected inputs we’re going to need annotations (the targets) for them. Sometimes, annotations can be retrieved automatically (music recommendation task, the click-through-rate prediction task), but often we have to annotate our data by hand. In this situation the following options are available:
    - annotate the data ourself
    - use a crowdsourcing platform like Mechanical Turk to collect labels
    - use the services of a specialized data-labeling company
2. **Representativeness** - Machine learning models can only make sense of inputs that are similar to what they’ve seen before. As such, it’s critical that the data used for training should be representative of the production data. This concern should be the foundation of all data collection work. Therefore, if possible, collect data directly from the environment where the model will be used. If it’s not possible to train on production data, then make sure you fully understand how your training and production data differ, and that you are actively correcting for these differences. A related phenomenon is concept drift. It occurs when the properties of the production data change over time, causing model accuracy to gradually decay. Dealing with fast concept drift requires constant data collection, annotation, and model retraining. Another particularly insidious and common case of non-representative data is sampling bias which occurs when  data collection process interacts with what we are trying to predict, resulting in biased measurements. 

### Understanding data
We shouldn't treat a dataset as a black box. Before we start training models, we should explore and visualize our data to gain insights about what makes it predictive, which will inform feature engineering and screen for potential issues. Moreover, if some samples are missing values for some features, we’ll need to deal with this when we prepare the data. If the classes roughly equally represented in a classification problem, we'll need to account for this imbalance. Finally, we need to check for target leaking: the presence of features in our data that provide information about the targets and which may not be available in production.

### Choosing a measure of success
To control something, we need to be able to observe it. To achieve success on a project, we must first define what you mean by success. Our metric for success will guide all of the technical choices we make throughout the project. It should directly align with our higher-level goals, such as the business success of our customer.

For balanced classification problems, where every class is equally likely, accuracy and the area under a receiver operating characteristic (ROC) curve [ROC AUC], are common metrics. For class-imbalanced problems, ranking problems, or multilabel classification, we can use precision and recall, as well as a weighted form of accuracy or ROC AUC. And it isn’t uncommon to have to define our own custom metric by which to measure success.

## Model development
When we know how we will measure our progress, we can develop a model. Model development is only one step in the machine learning workflow and it’s not the most difficult one. The hardest things in machine learning are framing problems and collecting, annotating, and cleaning data. At this stage we can specify the following steps:
- preparing the data
- choosing an evaluation protocol
- beating a baseline
- developping a model that overfits (scaling up)
- regularizing and tuning model

### Preparing the data
Deep learning models typically demand data preprocessing. This includes vectorization, normalization, or handling missing values. Many preprocessing techniques are domain-specific (for example, specific to text data or image data). 

#### Vectorization
All inputs and targets in a neural network must typically be tensors of floating-point data (or, in specific cases, tensors of integers or strings). This step is called data vectorization.

#### Value normalization
In general, it isn’t safe to feed into a neural network data that takes relatively large values (for example, multi-digit integers, which are much larger than the initial values taken by the weights of a network) or data that is heterogeneous (for example, data where one feature is in the range 0–1 and another is in the range 100–200). Doing so can trigger large gradient updates that will prevent the network from converging. To make learning easier for our network, our data should have the following characteristics:
- Take small values—Typically, most values should be in the 0–1 range.
- Be homogenous—All features should take values in roughly the same range.

Additionally, the following stricter normalization practice is common:
- Normalize each feature independently to have a mean of 0.
- Normalize each feature independently to have a standard deviation of 1.

This aim can be achieved using numpy
```python
x -= x.mean(axis=0)      
x /= x.std(axis=0)
```

#### Handling missing values
If you have missing values in your data you don’t necessarily have to discard the feature entirely:
- If the feature is categorical, it’s safe to create a new category that means “the value is missing.” The model will automatically learn what this implies with respect to the targets.
- If the feature is numerical, avoid inputting an arbitrary value like "0", because it may create a discontinuity in the latent space formed by your features, making it harder for a model trained on it to generalize. Instead, consider replacing the missing value with the average or median value for the feature in the dataset. We can also train a model to predict the feature value given the values of other features.


### Choosing an evaluation protocol
There are three common evaluation protocols:
- A holdout validation set — This is the way to go when we have plenty of data.
- $K$-fold cross-validation — This is the right choice when we have too few samples for holdout validation to be reliable.
- Iterated $K$-fold validation — This is for performing highly accurate model evaluation when little data is available.

### Beating a baseline
Our initial goal is to develop a small model that is capable of beating a simple baseline.
At this stage, these are the three most important things we should focus on:
- Feature engineering — Filter out uninformative features (feature selection) and use our knowledge of the problem to develop new features that are likely to be useful.
- Selecting the correct architecture priors — What type of model architecture will we use? A densely connected network, a convnet, a recurrent neural network, a Transformer? Is deep learning even a good approach for the task, or should we use something else?
- Selecting a good-enough training configuration — What loss function should we use? What batch size and learning rate?

It’s often not possible to directly optimize for the metric that measures success on a problem. Sometimes there is no easy way to turn a metric into a loss function. Loss functions need to be computable given only a mini-batch of data and must be differentiable. Otherwise, we can’t use backpropagation to train your network. For some common problem types the choice of a last-layer activation and a loss function can be as follows:

<img src=1.png/>

For most problems, there are existing templates we can start from.

### Scaling up
The ideal model is one that stands right at the border between underfitting and overfitting. To figure out where this border lies, first we must cross it. To figure out how big a model we’ll need, we must develop a model that overfits. This can be 
reached by
- adding layers
- making the layers bigger
- increasing the number of epochs.

We should always monitor the training loss and validation loss, as well as the training and validation values for any metrics we care about. When you see that the model’s performance on the validation data begins to degrade, we’ve achieved overfitting.

### Regularizing and tuning model
The maximization of generalization will take the most time: we’ll repeatedly modify our model, train it, evaluate on our validation data, modify it again, and repeat, until the model is as good as it can get. As we have discussed this issue in detail at the previous lecture. In particular, we can:
- Try different hyperparameters (such as the number of units per layer or the learning rate of the optimizer) to find the optimal configuration.
- Try different architectures; add or remove layers.
- If a model is small, add L1 or L2 regularization.
- Add dropout.
- Optionally, iterate on data curation or feature engineering: collect and annotate more data, develop better features, or remove features that don’t seem to be informative.

It’s possible to automate a large chunk of this work by using automated hyperparameter tuning software, such as KerasTuner. 

We need to keep in mind that every time we use feedback from your validation process to tune our model, we leak information about the validation process into the model. Repeated just a few times, this is innocuous; done systematically over many iterations, it will eventually cause our model to overfit to the validation process. This makes the evaluation process less reliable.

Once we’ve developed a satisfactory model configuration, we can train our final production model on all the available data (training and validation) and evaluate it one last time on the test set. If it turns out that performance on the test set is significantly worse than the performance measured on the validation data, this may mean either that our validation procedure wasn’t reliable after all, or that we began overfitting to the validation data while tuning the parameters of the model. In this case, we may need to use a more reliable evaluation protocol such as iterated $K$-fold validation.

## Deployong the model
### Explain your work to stakeholders and set expectations
The expectations of non-specialists towards AI systems are often unrealistic. For example, they might expect that the system “understands” its task and is capable of exercising human-like common sense in the context of the task. To address this, we should consider showing some examples of the failure modes of our model. 

They might also expect human-level performance, especially for processes that were previously handled by people. Most machine learning models do not nearly get there. We should clearly convey model performance expectations and avoid using abstract statements. 

We should also discuss with stakeholders the choice of key launch parameters—for instance, the probability threshold at which a transaction should be flagged (different thresholds will produce different false negative and false positive rates). Such decisions involve trade-offs that can only be handled with a deep understanding of the business context.

### Ship an inference model
We rarely put in production the exact same Python model object that you manipulated during training. First, we may want to export our model to something other than Python. Second, since our production model will only be used to output predictions (inference), we have room to perform various optimizations that can make the model faster and reduce its memory footprint. There are available different model deployment options:

#### DEPLOYING A MODEL AS A REST API
This is perhaps the common way to turn a model into a product: install TensorFlow on a server or cloud instance, and query the model’s predictions via a REST API. We can build our own serving app using TensorFlow’s own library for shipping models as APIs, called TensorFlow Serving (www.tensorflow.org/tfx/guide/serving).
We should use this deployment setup when:
- The application that will consume the model’s prediction will have reliable access to the internet.
- The application does not have strict latency requirements: the request, inference, and answer round trip will typically take around 500 ms.
- The input data sent for inference is not highly sensitive.

For instance, the image search engine project, the music recommender system, the credit card fraud detection project, and the satellite imagery project are all good fits for serving via a REST API.

#### DEPLOYING A MODEL ON A DEVICE
Sometimes, we may need our model to live on the same device that runs the application that uses it. We should use this setup when
- Our model has strict latency constraints or needs to run in a low-connectivity environment.
- Our model can be made sufficiently small that it can run under the memory and power constraints of the target device. We can use the TensorFlow Model Optimization Toolkit to help with this (www.tensorflow.org/model_optimization).
- Getting the highest possible accuracy isn’t mission critical for our task. There is always a trade-off between runtime efficiency and accuracy, so memory and power constraints often require us to ship a model that isn’t quite as good as the best model we could run on a large GPU.
- The input data is strictly sensitive and thus shouldn’t be decryptable on a remote server.

To deploy a Keras model on a smartphone or embedded device, a go-to solution is TensorFlow Lite (www.tensorflow.org/lite). It’s a framework for efficient on-device deep learning inference that runs on Android and iOS smartphones, as well as ARM64-based computers, Raspberry Pi, or certain microcontrollers.

#### DEPLOYING A MODEL IN THE BROWSER
Deep learning is often used in browser-based or desktop-based JavaScript applications when:
- We want to offload compute to the end user, which can dramatically reduce server costs.
- The input data needs to stay on the end user’s computer or phone. 
- Our application has strict latency constraints. 
- We need our app to keep working without connectivity, after the model has been downloaded and cached.

To deploy a model in JavaScript, the TensorFlow ecosystem includes TensorFlow.js (www.tensorflow.org/js).

### Monitor your model in the wild
Once we’ve exported an inference model, we’ve integrated it into our application, and we’ve done a dry run on production data—the model behaved exactly as we expected we can deploy it to production. However, this is not the end. We need to keep monitoring its behavior, its performance on new data, its interaction with the rest of the application, and its eventual impact on business metrics.

### Maintain your model
No model lasts forever. Over time, the characteristics of your production data will change, gradually degrading the performance and relevance of your model. A couple of years in the best case for the image search engine. As soon as your model has launched, you should be getting ready to train the next generation that will replace it. As such,
- Watch out for changes in the production data. 
- Keep collecting and annotating data, and keep improving your annotation pipeline over time. In particular, you should pay special attention to collecting samples that seem to be difficult for your current model to classify—such samples are the most likely to help improve performance.